In [1]:
from __future__ import absolute_import, division, print_function

In [2]:
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10

/Users/ankurkothari/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/ankurkothari/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

In [4]:
print(tf.__version__)
print(np.__version__)
print(matplotlib.__version__)

1.6.0-dev20180115
1.14.0
2.1.0


In [5]:
(features, labels), (X_test_orig, Y_test_orig) = cifar10.load_data()
features = features/255 - 0.5
labels = labels.flatten()

test_images = X_test_orig/255 - 0.5
test_labels = Y_test_orig.flatten()


In [6]:
features[0].shape

(32, 32, 3)

In [7]:
labels = labels.flatten()
labels

array([6, 9, 9, ..., 9, 1, 1], dtype=uint8)

In [8]:
height = 32
width = 32
channels = 3
n_inputs = height * width

In [9]:
tf.reset_default_graph()

In [10]:
def initialize_parameters():
    height = 32
    width = 32
    channels = 3
    n_inputs = height * width
    X = tf.placeholder(tf.float32, shape=[None,  height, width, channels], name="X")
    
    y = tf.placeholder(tf.int32, shape=[None], name="y")
    return X, y

In [11]:
def get_next_batch(features, labels, train_size, batch_index, batch_size):
    training_images = features[:train_size,:,:]
    training_labels = labels[:train_size]
    
    start_index = batch_index * batch_size
    end_index = start_index + batch_size

    return features[start_index:end_index,:,:], labels[start_index:end_index]

In [12]:
def forward_propagation(X, training=True):
  
    # Convolutional Layer #1
    dropout_rate = 0.3

    training = tf.placeholder_with_default(False, shape=(), name='training')
    X_drop = tf.layers.dropout(X, dropout_rate, training=training)
    print(X_drop)
    conv1 = tf.layers.conv2d(
      inputs=X_drop,
      filters=32,
      kernel_size=[5, 5],
      padding="valid",
      activation=tf.nn.relu)

    conv2 = tf.layers.conv2d(conv1, filters=64, 
                       kernel_size=3,
                       strides=2, padding="valid",
                       activation=tf.nn.relu, name="conv2")
    pool3 = tf.nn.max_pool(conv2,
                     ksize=[1, 2, 2, 1],
                     strides=[1, 2, 2, 1],
                     padding="VALID")
    conv4 = tf.layers.conv2d(pool3, filters=128, 
                       kernel_size=4,
                       strides=3, padding="SAME",
                       activation=tf.nn.relu, name="conv4")

    pool5 = tf.nn.max_pool(conv4,
                     ksize=[1, 2, 2, 1],
                     strides=[1, 1, 1, 1],
                     padding="VALID")

    pool5_flat = tf.contrib.layers.flatten(pool5)

    fullyconn1 = tf.layers.dense(pool5_flat, 128,
                           activation=tf.nn.relu, name="fc1")

    fullyconn2 = tf.layers.dense(fullyconn1, 64,
                           activation=tf.nn.relu, name="fc2")
    logits = tf.layers.dense(fullyconn2, 10, name="output")
    return logits, training

In [ ]:
tf.logging.set_verbosity(tf.logging.INFO)

def input_fn():
    
    data = [[[1, 0, 0, 0] for i in range(10)] for _ in range(100)]
    label = [[1] for i in range(100)]
    x = tf.constant(features, tf.float32)
    y_ = tf.constant(labels, tf.int32)
    return x, y_

In [ ]:
def model_fn(features, labels, mode):
    conv1 = tf.layers.conv2d(
      inputs=features,
      filters=32,
      kernel_size=[5, 5],
      padding="valid",
      activation=tf.nn.relu)

    conv2 = tf.layers.conv2d(conv1, filters=64, 
                       kernel_size=3,
                       strides=2, padding="valid",
                       activation=tf.nn.relu, name="conv2")
    pool3 = tf.nn.max_pool(conv2,
                     ksize=[1, 2, 2, 1],
                     strides=[1, 2, 2, 1],
                     padding="VALID")
    conv4 = tf.layers.conv2d(pool3, filters=128, 
                       kernel_size=4,
                       strides=3, padding="SAME",
                       activation=tf.nn.relu, name="conv4")

    pool5 = tf.nn.max_pool(conv4,
                     ksize=[1, 2, 2, 1],
                     strides=[1, 1, 1, 1],
                     padding="VALID")

    pool5_flat = tf.contrib.layers.flatten(pool5)

    fullyconn1 = tf.layers.dense(pool5_flat, 128,
                           activation=tf.nn.relu, name="fc1")

    fullyconn2 = tf.layers.dense(fullyconn1, 64,
                           activation=tf.nn.relu, name="fc2")
    logits = tf.layers.dense(fullyconn2, 10, name="output")
    global_step = tf.train.get_global_step()
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,
                                                      labels=labels)
    loss = tf.reduce_mean(xentropy)
    tf.summary.scalar("loss", loss)
    train_op = tf.train.GradientDescentOptimizer(1e-2).minimize(loss, global_step)
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)


est = tf.estimator.Estimator(model_fn)
est.train(input_fn, steps=1)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/z0/7mvknc852v16yt6j7nhtjscc0000gn/T/tmpl707gu46', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10b417630>, '_task_type': 'worker', '_task_id': 0, '_global_id': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [158]:
tf.reset_default_graph()

n_epochs = 1
batch_size = 128
X, y = initialize_parameters()
logits, training = forward_propagation(X)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,
                                                      labels=y)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32)) 

Tensor("dropout/cond/Merge:0", shape=(?, 32, 32, 3), dtype=float32)


In [159]:
def model(features, labels, test_images, test_labels):
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        train_writer = tf.summary.FileWriter( './logs/1/train ', sess.graph)
        sess.run(init)
        for epoch in range(n_epochs):
    #         batch_index = 0
            # Add this in when we want to run the training on all batches in CIFAR-10
            batch_index = 0

            train_size = int(len(features))

            for iteration in range(train_size // batch_size):
                X_batch, y_batch = get_next_batch(features, 
                                                                            labels, 
                                                                            train_size, 
                                                                            batch_index,
                                                                            batch_size)
                batch_index += 1

                sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})

            acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
            acc_test = accuracy.eval(feed_dict={X: test_images, y: test_labels})
            print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
            print ("Train Accuracy:", round(acc_train, 2)*100,"%",  end="\t")
            print ("Test Accuracy:", round(acc_test, 2)*100,"%", end="\n")
        import os
        if not os.path.exists("ADL"):
            os.makedirs("ADL")
        model_saver = tf.train.Saver()
        save_path = model_saver.save(sess, "ADL/cifar_model")

In [160]:
model(features, labels, test_images, test_labels)

0 Train accuracy: 0.546875 Test accuracy: 0.4964
Train Accuracy: 55.000001192092896 %	Test Accuracy: 50.0 %


In [179]:
import tensorflow as tf
from tensorflow.python.tools import inspect_checkpoint as chkp

tf.reset_default_graph()
with tf.Session() as sess:
    # Restore variables from disk.
    print(sess.graph_def)
    saver = tf.train.import_meta_graph('ADL/cifar_model.meta')
    saver.restore(sess,"ADL/cifar_model")
    
    graph = tf.get_default_graph()
    print(sess.graph_def)
    for var in tf.train.list_variables("ADL/cifar_model"):
        print(var)
    from tensorflow.python.tools import inspect_checkpoint as chkp

# print all tensors in checkpoint file
    chkp.print_tensors_in_checkpoint_file("ADL/cifar_model", tensor_name='', all_tensors=True, all_tensor_names = True)
    out = tf.get_default_graph().get_tensor_by_name("output/kernel/Adam_1")
    sess.run(out)
    

versions {
  producer: 25
}

INFO:tensorflow:Restoring parameters from ADL/cifar_model
node {
  name: "X"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
    }
  }
}
node {
  name: "y"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
node {
  name: "training/input"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_BOOL
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_BOOL
        tensor_shape {
        }
        bool_val: false
      }
    }
  }
}
node {
  name: "training"
  op: "PlaceholderWithDefault"
  input: "train

ValueError: The name 'output/kernel/Adam_1' refers to an Operation, not a Tensor. Tensor names must be of the form "<op_name>:<output_index>".

In [128]:
logits

<tf.Tensor 'output/BiasAdd:0' shape=(?, 10) dtype=float32>